In [1]:
import _config_mac_to_aws as _config

In [4]:
from go.bot.go_db import GoDB
from sqlmodel import Session, SQLModel, create_engine
from go.bot.playfab_api import PlayfabApi, as_player_id, as_playfab_id
from go.bot.playfab_db import PlayfabDB
from go.bot.models import GoTeam
from pprint import pprint

engine = create_engine(_config.godb_url, echo=_config.godb_echo)

SQLModel.metadata.create_all(engine)
pfdb = PlayfabDB()
godb = GoDB()

pfapi = PlayfabApi()
pfapi.login_to_playfab()



2024-06-14 13:45:30.157  INFO      go.bot.playfab_api -- login_to_playfab: session ticket set


# Choose a playfabid

In [3]:
stooobe_playfabid = '121559C6C39206B5'
lester_playfabid = 'B6395EBFD2459486'
alexfromc_playfabid = 'F9F21325130975C'
revolutionx_playfabid = '4575D3B047A8981E'
coffee_backon_playfabid = as_playfab_id(1065470778045992877)
bradwolf_playfabid = as_playfab_id(-119964107432152410)
provolown_pfid = '62D75D8FC9785191'

pfid = stooobe_playfabid

In [12]:
print(pfapi.get_player_from_account_info(player_id=as_player_id(pfid)))
print("stats:")
print(pfapi.get_player_career_stats(player_id=as_player_id(pfid)))

Player[Provolown, id -2101107829003234927, created 2021-11-26, last_login 2024-06-13 15:56]
stats:
date=datetime.datetime(2024, 6, 13, 15, 5, 13, 348381) pf_player_id=-2101107829003234927 games=5005 wins=2273 kills=19892 damage=5200175 mmr=4886 skill=2441


In [8]:
session = Session(engine)
Stoobe = godb.read_player(discord_id=408731638223208448, session=session)
godb.create_team('solo stooobe', [Stoobe], session, rating_limit=None, season=_config.go_season)

2024-06-14 13:48:34.119  INFO      go.bot.go_db   -- Reading GoPlayer with discord_id = 408731638223208448 from DB
2024-06-14 13:48:34.221  INFO      go.bot.go_db   -- Creating GoTeam team_name = 'solo stooobe' in DB
2024-06-14 13:48:34.221  INFO      go.bot.go_db   -- Reading GoTeam with discord_ids = {408731638223208448} from DB
2024-06-14 13:48:34.307  INFO      go.bot.go_db   -- Returning team with team.id = 70


GoDbError: Team with roster {'GO_STOOOBE'} already exists.

# Add an official GoRating

In [ ]:
from go.bot.models import GoRatings

with Session(engine) as session:

    player = pfdb.read_player(pf_player_id=as_player_id(pfid), session=session)
    print(f'{player =}')
    if player is None:
        raise Exception()

    career_stats = [_ for _ in player.career_stats]
    career_stats.sort(key=lambda x: x.date)
    pprint(career_stats)

    stats = career_stats[-1]
    print(f'{stats =}, rating={stats.calc_rating()}')

    go_rating = godb.get_official_rating(player.id, session, season=_config.go_season)
    print(f'{go_rating =}')
    
    if go_rating is None:
        official_rating = GoRatings(
            pf_player_id=player.id,
            season=_config.go_season,
            rating_type="official",
            go_rating=stats.calc_rating(),
            )
        session.add(official_rating)
        session.commit()


# Register with Playfab 
Ran once at beginning of project

In [ ]:
# headers = {
#     'Content-Type': 'application/json',
#     #'X-SecretKey': api_key, 
#     # 'X-Authorization': session_ticket
# }

# payload = {
#     "TitleId": title_id,
#     "Username": "devstooobe",  # Replace with the desired username
#     "Email": "brianstube@gmail.com",  # Replace with the user's email
#     "Password": "vabiewu2398fh"  # Replace with the user's password
# }

# response = run_request('RegisterPlayFabUser', payload, headers)